In [17]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import datetime
import os
from torch.utils.tensorboard import SummaryWriter
import shutil
import matplotlib.pyplot as plt
import numpy as np
from tqdm.auto import tqdm

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
transform_train = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomCrop(224, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4876, 0.4544, 0.4165), (0.2257, 0.2209, 0.2212)),
])

transform_test = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.4876, 0.4544, 0.4165), (0.2257, 0.2209, 0.2212)),
])

train_dataset = torchvision.datasets.ImageFolder(root = os.path.join(os.getcwd(), 'CatvsDog_train'), transform = transform_train)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = 128, shuffle = True, num_workers = 0)

test_dataset = torchvision.datasets.ImageFolder(root = os.path.join(os.getcwd(), 'CatvsDog_test'), transform = transform_test)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size = 100, shuffle = False, num_workers = 0)

In [4]:
print(test_dataset.class_to_idx)

{'Cat': 0, 'Dog': 1}


In [6]:
current_time = datetime.datetime.now() + datetime.timedelta(hours= 9)
current_time = current_time.strftime('%Y-%m-%d-%H:%M')

saved_loc = os.path.join('./', current_time)
if os.path.exists(saved_loc):
    shutil.rmtree(saved_loc)
os.mkdir(saved_loc)

print("결과 저장 위치: ", saved_loc)

writer = SummaryWriter(saved_loc)

결과 저장 위치:  ./2021-05-04-13:55


In [7]:
import torchvision.models as models

resnet18 = models.resnet18(pretrained = False)

print(resnet18.fc)
resnet18.fc = nn.Linear(512, 2)
print(resnet18.fc)

Linear(in_features=512, out_features=1000, bias=True)
Linear(in_features=512, out_features=2, bias=True)


In [8]:
print(resnet18)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [13]:
net = resnet18.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.1,
                      momentum=0.9, weight_decay=0.0005)
scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones = [60, 120, 160], gamma = 0.2)

best_acc = 0

In [14]:
# Training
def train(epoch):
    print('\nEpoch: %d' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    batch_count = 0
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        #print("inputs: ", inputs.shape)
        #print("targets: ", targets.shape)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        batch_count += 1

    print("Train Loss : {:.3f} | Train Acc: {:.3f}".format(train_loss / batch_count, 100.*correct/total))
    final_loss = train_loss / batch_count
    final_acc = 100.*correct / total
    return final_loss, final_acc


def test(epoch):
    global best_acc
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    batch_count = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(test_loader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
            batch_count += 1


    print("Test Loss : {:.3f} | Test Acc: {:.3f}".format(test_loss / batch_count, 100.*correct/total))

    # Save checkpoint.
    acc = 100.*correct/total
    if acc > best_acc:
        print('Saving..')
        state = {
            'net': net.state_dict(),
            'acc': acc,
            'epoch': epoch,
        }
        if not os.path.isdir(os.path.join(saved_loc, 'checkpoint')):
            os.mkdir(os.path.join(saved_loc, 'checkpoint'))
        torch.save(state, os.path.join(saved_loc, 'checkpoint/ckpt.pth'))
        best_acc = acc
    
    final_loss = test_loss / batch_count
    return final_loss, acc

In [15]:
for epoch in tqdm(range(200)):
    train_loss, train_acc = train(epoch)
    test_loss, test_acc = test(epoch)

    writer.add_scalars('Graphs/Loss', {"Train Loss" : train_loss, "Test Loss" : test_loss}, epoch)
    writer.add_scalars('Graphs/Accuracy', {"Train acc" : train_acc, "test acc" : test_acc}, epoch)
    
    scheduler.step()

writer.close()


Epoch: 0
Train Loss : 1.114 | Train Acc: 50.870
Test Loss : 0.699 | Test Acc: 52.280
Saving..

Epoch: 1
Train Loss : 0.678 | Train Acc: 58.465
Test Loss : 0.698 | Test Acc: 55.560
Saving..

Epoch: 2
Train Loss : 0.646 | Train Acc: 63.200
Test Loss : 0.648 | Test Acc: 62.960
Saving..

Epoch: 3
Train Loss : 0.617 | Train Acc: 66.225
Test Loss : 0.616 | Test Acc: 65.800
Saving..

Epoch: 4
Train Loss : 0.586 | Train Acc: 68.990
Test Loss : 0.573 | Test Acc: 69.340
Saving..

Epoch: 5
Train Loss : 0.555 | Train Acc: 71.415
Test Loss : 0.711 | Test Acc: 64.180

Epoch: 6
Train Loss : 0.527 | Train Acc: 73.765
Test Loss : 0.539 | Test Acc: 72.660
Saving..

Epoch: 7
Train Loss : 0.504 | Train Acc: 74.965
Test Loss : 0.510 | Test Acc: 74.540
Saving..

Epoch: 8
Train Loss : 0.481 | Train Acc: 76.570
Test Loss : 0.534 | Test Acc: 72.280

Epoch: 9
Train Loss : 0.453 | Train Acc: 78.895
Test Loss : 0.460 | Test Acc: 78.520
Saving..

Epoch: 10
Train Loss : 0.400 | Train Acc: 81.865
Test Loss : 0.409 

Test Loss : 0.150 | Test Acc: 95.520

Epoch: 93
Train Loss : 0.015 | Train Acc: 99.500
Test Loss : 0.139 | Test Acc: 95.920

Epoch: 94
Train Loss : 0.013 | Train Acc: 99.525
Test Loss : 0.140 | Test Acc: 95.600

Epoch: 95
Train Loss : 0.021 | Train Acc: 99.290
Test Loss : 0.116 | Test Acc: 96.260

Epoch: 96
Train Loss : 0.026 | Train Acc: 99.105
Test Loss : 0.126 | Test Acc: 96.160

Epoch: 97
Train Loss : 0.011 | Train Acc: 99.675
Test Loss : 0.127 | Test Acc: 96.540

Epoch: 98
Train Loss : 0.021 | Train Acc: 99.270
Test Loss : 0.157 | Test Acc: 95.540

Epoch: 99
Train Loss : 0.023 | Train Acc: 99.160
Test Loss : 0.124 | Test Acc: 96.080

Epoch: 100
Train Loss : 0.008 | Train Acc: 99.730
Test Loss : 0.134 | Test Acc: 96.140

Epoch: 101
Train Loss : 0.010 | Train Acc: 99.700
Test Loss : 0.134 | Test Acc: 96.140

Epoch: 102
Train Loss : 0.022 | Train Acc: 99.180
Test Loss : 0.134 | Test Acc: 95.920

Epoch: 103
Train Loss : 0.014 | Train Acc: 99.480
Test Loss : 0.147 | Test Acc: 95.820

E

Test Loss : 0.109 | Test Acc: 97.260

Epoch: 186
Train Loss : 0.001 | Train Acc: 100.000
Test Loss : 0.111 | Test Acc: 97.220

Epoch: 187
Train Loss : 0.001 | Train Acc: 99.995
Test Loss : 0.111 | Test Acc: 97.300

Epoch: 188
Train Loss : 0.001 | Train Acc: 99.990
Test Loss : 0.111 | Test Acc: 97.220

Epoch: 189
Train Loss : 0.001 | Train Acc: 99.995
Test Loss : 0.110 | Test Acc: 97.300

Epoch: 190
Train Loss : 0.001 | Train Acc: 99.995
Test Loss : 0.109 | Test Acc: 97.240

Epoch: 191
Train Loss : 0.001 | Train Acc: 99.990
Test Loss : 0.111 | Test Acc: 97.200

Epoch: 192
Train Loss : 0.001 | Train Acc: 99.990
Test Loss : 0.110 | Test Acc: 97.260

Epoch: 193
Train Loss : 0.001 | Train Acc: 99.990
Test Loss : 0.108 | Test Acc: 97.260

Epoch: 194
Train Loss : 0.001 | Train Acc: 99.995
Test Loss : 0.108 | Test Acc: 97.260

Epoch: 195
Train Loss : 0.001 | Train Acc: 99.990
Test Loss : 0.107 | Test Acc: 97.180

Epoch: 196
Train Loss : 0.001 | Train Acc: 99.995
Test Loss : 0.110 | Test Acc: 9

In [20]:
ckpt = torch.load(os.path.join(os.getcwd(), '2021-05-04-13:55/checkpoint/ckpt.pth'))

print(ckpt['acc'])
print(ckpt['epoch'])

97.32
145
